In [1]:
import Minkowski as mn
import AdS as ads
from mm_estimator import mm_dimension
from longest_chain import *
from interval import filter_interval
import matplotlib.pyplot as plt
import numpy as np
import scipy.optimize as op
%matplotlib tk
from importlib import reload

params= {
   'axes.labelsize': 16,
   'font.size': 18,
   'legend.fontsize': 18,
   'xtick.labelsize': 16,
   'ytick.labelsize': 16,
   'figure.figsize': [8, 6] 
   }
plt.rcParams.update(params)

def scatter(position,x=0,y=1):
    plt.scatter(position[:,x],position[:,y],s=1)

def plot_interval(l,interval_centre,half=True):
   R=interval_centre[0]
   t=interval_centre[1]
   if half==True:
      plt.plot([R,R+l/2,R],[t+l/2,t,t-l/2])
   else:
      plt.plot([R,R+l/2,R,R-l/2,R],[t+l/2,t,t-l/2,t,t+l/2])

def plot_link(position,link,xindex=0,yindex=1):
    for i in link:
        for j in link[i]:
            plt.plot([position[i,xindex],position[j,xindex]],[position[i,yindex],position[j,yindex]],color='b')

def fitting_func1(Nintervallist,md,d):
    y=[]
    for i in Nintervallist:
        y.append(md*i**(1/d))
    return y

def fitting_func2(Nintervallist,md,d,A):
    y=[]
    for i in Nintervallist:
        y.append(md*i**(1/d)+A)
    return y

In [132]:
#plotting links on minkowski space
N=50
L=1
d=2
interval_centre=np.zeros(d)
position=mn.sprinkle(N,L,d,False)
link=find_link(position)
scatter(position)
plot_link(position,link)
plt.title(r'Sprinkling into Minkowski space with links plotted on at $\langle N\rangle$=%s L=%s d=%s'%(N,L,d),fontsize=13)
plt.xlabel('R')
plt.ylabel('t')
plt.show()

In [6]:
#testing mm estimator
N=1000
L=1
d=2
interval_centre=np.zeros(d)
position=mn.sprinkle(N,L,d,False)
position_subset=filter_interval(position,L,interval_centre)
relation=find_relation(position_subset)
Nrel=sum(len(relation[i]) for i in range(len(relation)))
frac=Nrel/len(position_subset)**2
dim=mm_dimension(frac)
print(dim)

1.991343451769964
0.9913434517699644


In [3]:
#plotting MM estimator function
from mm_estimator import ordering_frac
dlist=np.linspace(0,20,1000)
frac=ordering_frac(dlist)

fig,ax=plt.subplots()
ax.yaxis.set_label_coords(-0.1,0)
plt.title('Ordering fraction as a function of d')
plt.xlabel('d')
plt.ylabel(r'$\frac{\langle R\rangle}{\langle N\rangle ^2}$',rotation=0,fontsize=20)
plt.plot(dlist,frac)
plt.grid()
plt.show()

In [143]:
#Sprinkle into AdS space
N=5000
L=1
d=4
R_0=0.1
interval_centre=[R_0,0]
for i in range(d-2):
    interval_centre.append(0)
position=ads.sprinkle(N,L,d,R_0,False,True)
position_subset=filter_interval(position,L,interval_centre)
plt.title(r'Sprinkling into AdS space at $\langle N\rangle$=%s L=%s d=%s $R_0$=%s' %(N,L,d,R_0),fontsize=15)
plt.xlabel('R')
plt.ylabel('t')
scatter(position)
scatter(position_subset)
plot_interval(L,interval_centre,True)
plt.legend(['Sprinkled points','Points inside interval','interval bounds'],fontsize=15)
plt.show()

In [82]:
#investigate full interval as function of R_0
N=20000
L=1000
d=2
R_0list=np.linspace(0.05,1,20)

discretenesslist=[]
dimlist=[]
dimstd=[]
for R_0 in R_0list:
    interval_centre=[R_0+L/2,0]
    for i in range(d-2):
        interval_centre.append(0)

    V=L**(d-1)/(d-1)*(R_0**(1-d)-(R_0+L)**(1-d))
    densityads=N/V
    discreteness=densityads**(-1/d)
    discretenesslist.append(discreteness)

    Nrelsub=[]
    Nintervalsub=[]
    for count1 in range(20):
        position=ads.sprinkle(N,L,d,R_0,False,False)
        position_subset=filter_interval(position,L,interval_centre)
        relation=find_relation(position_subset)
        Nrel=sum(len(relation[i]) for i in range(len(relation)))
        Ninterval=len(position_subset)
        Nrelsub.append(Nrel)
        Nintervalsub.append(Ninterval)
        print(count1)
    Nrelmean=np.mean(Nrelsub)
    Nrelstd=np.std(Nrelsub)
    Nintervalmean=np.mean(Nintervalsub)
    Nintervalstd=np.std(Nintervalsub)
    fracmean=Nrelmean/Nintervalmean**2
    fracstd=np.sqrt((Nrelstd/Nintervalmean**2)**2+(Nintervalstd*2*Nrelmean/Nintervalmean**3)**2)
    upperdim=mm_dimension(fracmean+fracstd)
    lowerdim=mm_dimension(fracmean-fracstd)
    dimlist.append(np.mean([upperdim,lowerdim]))
    dimstd.append(np.std([upperdim,lowerdim]))
    print(R_0)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
0.05
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
0.1
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
0.15
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
0.2
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
0.25
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
0.3
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
0.35
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
0.39999999999999997
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
0.44999999999999996
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
0.49999999999999994
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
0.5499999999999999
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
0.6
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
0.65
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
0.7
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
0.75
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
0.7999999999999999
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
0.

In [117]:
plt.title(r'Discreteness length for interval at $R=R_0+L/2$;$\langle N\rangle$=%s d=%s L=%s'%(N,d,L),fontsize=15)
plt.xlabel(r'$R_0$')
plt.ylabel(r'$l_{discrete}$')
plt.plot(R_0list,discretenesslist)
plt.grid()
plt.show()

In [85]:
plt.title(r'Myrheim-Meyer dimension for interval at $R=R_0+L/2$;$\langle N\rangle$=%s d=%s L=%s'%(N,d,L),fontsize=15)
plt.xlabel(r'$R_0$')
plt.ylabel('Myrheim-Meyer dimension')
plt.errorbar(R_0list,dimlist,yerr=dimstd)
plt.grid()
plt.show()

In [75]:
#investigate full interval as function of R_0
N=20000
L=1000
d=2
R_0list=np.logspace(-2,0,20)

discretenesslist=[]
dimlist=[]
dimstd=[]
for R_0 in R_0list:
    interval_centre=[R_0+L/2,0]
    for i in range(d-2):
        interval_centre.append(0)

    V=L**(d-1)/(d-1)*(R_0**(1-d)-(R_0+L)**(1-d))
    densityads=N/V
    discreteness=densityads**(-1/d)
    discretenesslist.append(discreteness)

    Nrelsub=[]
    Nintervalsub=[]
    for count1 in range(20):
        position=ads.sprinkle(N,L,d,R_0,False,False)
        position_subset=filter_interval(position,L,interval_centre)
        relation=find_relation(position_subset)
        Nrel=sum(len(relation[i]) for i in range(len(relation)))
        Ninterval=len(position_subset)
        Nrelsub.append(Nrel)
        Nintervalsub.append(Ninterval)
        print(count1)
    Nrelmean=np.mean(Nrelsub)
    Nrelstd=np.std(Nrelsub)
    Nintervalmean=np.mean(Nintervalsub)
    Nintervalstd=np.std(Nintervalsub)
    fracmean=Nrelmean/Nintervalmean**2
    fracstd=np.sqrt((Nrelstd/Nintervalmean**2)**2+(Nintervalstd*2*Nrelmean/Nintervalmean**3)**2)
    upperdim=mm_dimension(fracmean+fracstd)
    lowerdim=mm_dimension(fracmean-fracstd)
    dimlist.append(np.mean([upperdim,lowerdim]))
    dimstd.append(np.std([upperdim,lowerdim]))
    print(R_0)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
0.01
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
0.016681005372000592
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
0.027825594022071243
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
0.046415888336127774
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
0.0774263682681127
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
0.1291549665014884
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
0.21544346900318834
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
0.3593813663804626
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
0.5994842503189409
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
1.0


In [66]:
plt.title(r'Discreteness length for interval at $R=R_0+L/2$;$\langle N\rangle$=%s d=%s L=%s'%(N,d,L),fontsize=15)
plt.xlabel(r'$R_0$')
plt.ylabel(r'$l_{discrete}$')
plt.xscale('log')
plt.plot(R_0list,discretenesslist)
plt.grid()
plt.show()

In [76]:
plt.title(r'Myrheim-Meyer dimension for interval at $R=R_0+L/2$;$\langle N\rangle$=%s d=%s L=%s'%(N,d,L),fontsize=15)
plt.xlabel(r'$R_0$')
plt.ylabel('Myrheim-Meyer dimension')
plt.xscale('log')
plt.errorbar(R_0list,dimlist,yerr=dimstd)
plt.grid()
plt.show()

In [48]:
#investigate full interval as function of d
N=10000
L=1
dlist=np.arange(3,9)
R_0=10000

discretenesslist=[]
dimlist=[]
dimstd=[]
for d in dlist:
    interval_centre=[R_0+L/2,0]
    for i in range(d-2):
        interval_centre.append(0)
    d=int(d)
    V=L**(d-1)/(d-1)*(R_0**(1-d)-(R_0+L)**(1-d))
    densityads=N/V
    discreteness=densityads**(-1/d)
    discretenesslist.append(discreteness)

    Nrelsub=[]
    Nintervalsub=[]
    for count1 in range(10):
        position=ads.sprinkle(N,L,d,R_0,False,False)
        position_subset=filter_interval(position,L,interval_centre)
        relation=find_relation(position_subset)
        Nrel=sum(len(relation[i]) for i in range(len(relation)))
        Ninterval=len(position_subset)
        Nrelsub.append(Nrel)
        Nintervalsub.append(Ninterval)
    Nrelmean=np.mean(Nrelsub)
    Nrelstd=np.std(Nrelsub)
    Nintervalmean=np.mean(Nintervalsub)
    Nintervalstd=np.std(Nintervalsub)
    fracmean=Nrelmean/Nintervalmean**2
    fracstd=np.sqrt((Nrelstd/Nintervalmean**2)**2+(Nintervalstd*2*Nrelmean/Nintervalmean**3)**2)
    upperdim=mm_dimension(fracmean+fracstd)
    lowerdim=mm_dimension(fracmean-fracstd)
    dimlist.append(np.mean([upperdim,lowerdim]))
    dimstd.append(np.std([upperdim,lowerdim]))
    print(d)

3
4
5
6
7
8


In [51]:
plt.title(r'Discreteness length for interval at $R=R_0+L/2$;$\langle N\rangle$=%s $R_0$=%s L=%s'%(N,R_0,L),fontsize=15)
plt.xlabel(r'AdS sprinkled dimension')
plt.ylabel(r'$l_{discrete}$')
plt.plot(dlist,discretenesslist)
plt.grid()
plt.show()


In [50]:
plt.title('Myrheim-Meyer dimension for interval at $R=R_0+L/2$;$\langle N\rangle$=%s $R_0$=%s L=%s'%(N,R_0,L),fontsize=15)
plt.xlabel(r'AdS sprinkled dimension')
plt.ylabel('Myrheim-Meyer dimension')
plt.errorbar(dlist,dimlist,yerr=dimstd)
plt.grid()
plt.show()

In [155]:
#investigate half interval as function of R_0
N=1000
L=0.001
d=2
R_0list=np.logspace(-7,-1,20)

discretenesslist=[]
dimlist=[]
dimstd=[]
for R_0 in R_0list:
    interval_centre=[R_0,0]
    for i in range(d-2):
        interval_centre.append(0)
    d=int(d)
    V=L**(d-1)/(d-1)*(R_0**(1-d)-(R_0+L/2)**(1-d))
    densityads=N/V
    discreteness=densityads**(-1/d)
    discretenesslist.append(discreteness)

    Nrelsub=[]
    Nintervalsub=[]
    for count1 in range(20):
        position=ads.sprinkle(N,L,d,R_0,False,True)
        position_subset=filter_interval(position,L,interval_centre)
        relation=find_relation(position_subset)
        Nrel=sum(len(relation[i]) for i in range(len(relation)))
        Ninterval=len(position_subset)
        Nrelsub.append(Nrel)
        Nintervalsub.append(Ninterval)
        print(count1)
    Nrelmean=np.mean(Nrelsub)
    Nrelstd=np.std(Nrelsub)
    Nintervalmean=np.mean(Nintervalsub)
    Nintervalstd=np.std(Nintervalsub)
    fracmean=Nrelmean/Nintervalmean**2
    fracstd=np.sqrt((Nrelstd/Nintervalmean**2)**2+(Nintervalstd*2*Nrelmean/Nintervalmean**3)**2)
    upperdim=mm_dimension(fracmean+fracstd)
    lowerdim=mm_dimension(fracmean-fracstd)
    dimlist.append(np.mean([upperdim,lowerdim]))
    dimstd.append(np.std([upperdim,lowerdim]))
    print(R_0)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
1e-07
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
2.06913808111479e-07
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
4.281332398719396e-07
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
8.858667904100833e-07
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
1.8329807108324375e-06
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
3.7926901907322535e-06
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
7.847599703514607e-06
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
1.623776739188721e-05
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
3.359818286283781e-05
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
6.951927961775606e-05
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
0.0001438449888287663
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
0.0002976351441631316
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
0.0006158482110660261
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
0.0012742749857031321
0
1
2
3

In [157]:
plt.title(r'Discreteness length for interval at $R=R_0+L/2$;$\langle N\rangle$=%s $d$=%i L=%s'%(N,d,L),fontsize=15)
plt.xlabel(r'$R_0$')
plt.ylabel(r'$l_{discrete}$')
plt.xscale('log')
plt.plot(R_0list,discretenesslist)
plt.grid()
plt.show()

In [159]:
plt.title(r'Myrheim-Meyer dimension for interval at $R=R_0+L/2$;$\langle N\rangle$=%s $d$=%i L=%s'%(N,d,L),fontsize=15)
plt.xlabel(r'$R_0$')
plt.ylabel('Myrheim-Meyer dimension')
plt.xscale('log')
plt.errorbar(R_0list,dimlist,yerr=dimstd)
plt.grid()
plt.show()

In [160]:
#investigate half interval as function of d
N=10000
L=1
dlist=np.arange(3,7)
R_0=0.0001

discretenesslist=[]
dimlist=[]
dimstd=[]
for d in dlist:
    interval_centre=[R_0,0]
    for i in range(d-2):
        interval_centre.append(0)
    d=int(d)
    V=L**(d-1)/(d-1)*(R_0**(1-d)-(R_0+L/2)**(1-d))
    densityads=N/V
    discreteness=densityads**(-1/d)
    discretenesslist.append(discreteness)

    Nrelsub=[]
    Nintervalsub=[]
    for count1 in range(20):
        position=ads.sprinkle(N,L,d,R_0,False,True)
        position_subset=filter_interval(position,L,interval_centre)
        relation=find_relation(position_subset)
        Nrel=sum(len(relation[i]) for i in range(len(relation)))
        Ninterval=len(position_subset)
        Nrelsub.append(Nrel)
        Nintervalsub.append(Ninterval)
        print(count1)
    Nrelmean=np.mean(Nrelsub)
    Nrelstd=np.std(Nrelsub)
    Nintervalmean=np.mean(Nintervalsub)
    Nintervalstd=np.std(Nintervalsub)
    fracmean=Nrelmean/Nintervalmean**2
    fracstd=np.sqrt((Nrelstd/Nintervalmean**2)**2+(Nintervalstd*2*Nrelmean/Nintervalmean**3)**2)
    upperdim=mm_dimension(fracmean+fracstd)
    lowerdim=mm_dimension(fracmean-fracstd)
    dimlist.append(np.mean([upperdim,lowerdim]))
    dimstd.append(np.std([upperdim,lowerdim]))
    print(d)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
3
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
4
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
5
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
6


In [162]:
plt.title(r'Discreteness length for interval at $R=R_0$;$\langle N\rangle$=%s $R_0$=%s L=%s'%(N,R_0,L),fontsize=15)
plt.xlabel(r'AdS sprinkled dimension')
plt.ylabel(r'$l_{discrete}$')
plt.plot(dlist,discretenesslist)
plt.grid()
plt.show()


In [161]:
plt.title(r'Myrheim-Meyer dimension for interval at $R=R_0$;$\langle N\rangle$=%s $R_0$=%s L=%s'%(N,R_0,L),fontsize=15)
plt.xlabel(r'AdS sprinkled dimension')
plt.ylabel('Myrheim-Meyer dimension')
plt.errorbar(dlist,dimlist,yerr=dimstd)
plt.grid()
plt.show()

In [69]:
#longest chain scaling for constant R0
Nlist=np.linspace(100,1000,20)
L=1
d=3
R_0=0.0000001
interval_centre=[R_0,0]
for i in range(d-2):
    interval_centre.append(0)
discretenesslist=[]
Nintervallist=[]
Nintervalstd=[]
lengthlist=[]
lengthstd=[]
for N in Nlist:
    V=L**(d-1)/(d-1)*(R_0**(1-d)-(R_0+L/2)**(1-d))
    densityads=N/V
    discreteness=densityads**(-1/d)
    discretenesslist.append(discreteness)
    Nintervalsub=[]
    lengthsub=[]
    for count1 in range(20):
        position=ads.sprinkle(N,L,d,R_0,False,True)
        position_subset=filter_interval(position,L,interval_centre)
        link=find_link(position_subset)
        longest=longest_chain(link)
        length=len(longest)
        lengthsub.append(length)
        Ninterval=len(position_subset)
        Nintervalsub.append(Ninterval)
        print(count1)
    Nintervallist.append(np.mean(Nintervalsub))
    Nintervalstd.append(np.std(Nintervalsub))
    lengthlist.append(np.mean(lengthsub))
    lengthstd.append(np.std(lengthsub))
    print(N)



0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
100.0
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
147.3684210526316
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
194.73684210526318
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
242.10526315789474
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
289.47368421052636
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
336.8421052631579
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
384.2105263157895
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
431.5789473684211
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
478.94736842105266
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
526.3157894736842
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
573.6842105263158
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
621.0526315789474
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
668.421052631579
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
715.7894736842105
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
763.157

In [37]:
plt.title(r'Discreteness length for interval at $R=R_0$; d=%s $R_0$=%s L=%s'%(d,R_0,L),fontsize=15)
plt.xlabel(r'$\left\langle N_{interval}\right\rangle$')
plt.ylabel(r'$l_{discrete}$')
plt.errorbar(Nintervallist,discretenesslist,xerr=Nintervalstd)
plt.grid()
plt.show()


In [74]:
fit=op.curve_fit(fitting_func2,Nintervallist,lengthlist,p0=[2,d-1,0],sigma=lengthstd)
print('m2=',fit[0][0],',d=',fit[0][1],',A=',fit[0][2])
plt.title(r'Longest chain length scaling against $\left\langle N_{interval}\right\rangle$; d=%s, $R_0$=%s, L=%s'%(d,R_0,L),fontsize=15)
plt.xlabel(r'$\left\langle N_{interval}\right\rangle$')
plt.ylabel('Length of longest chain')
plt.plot(Nintervallist,fitting_func2(Nintervallist,*fit[0]))
plt.errorbar(Nintervallist,lengthlist,yerr=lengthstd,xerr=Nintervalstd)
plt.grid()
plt.legend(['d=%.2f+-%.2f'%(fit[0][1],fit[1][1,1])])
plt.show()

m2= 1.1645363798293842 ,d= 1.7648707608570637 ,A= 0.6362145199833817


In [192]:
#longest chain scaling for different R0
Nlist=np.linspace(100,800,15)
L=1
d=2
R_0list=np.logspace(-5,5,20)
dlist=[]
dstd=[]
mlist=[]
mstd=[]
Alist=[]
Astd=[]
discretenesslist=[]
for R_0 in R_0list:
    interval_centre=[R_0,0]
    for i in range(d-2):
        interval_centre.append(0)
    V=L**(d-1)/(d-1)*(R_0**(1-d)-(R_0+L/2)**(1-d))
    Nintervallist=[]
    # Nintervalstd=[]
    lengthlist=[]
    lengthstd=[]
    for N in Nlist:
        densityads=N/V
        discreteness=densityads**(-1/d)
        discretenesslist.append(discreteness)
        Nintervalsub=[]
        lengthsub=[]
        for count1 in range(20):
            position=ads.sprinkle(N,L,d,R_0,False,True)
            position_subset=filter_interval(position,L,interval_centre)
            link=find_link(position_subset)
            longest=longest_chain(link)
            length=len(longest)
            lengthsub.append(length)
            Ninterval=len(position_subset)
            Nintervalsub.append(Ninterval)
            print(count1)
        Nintervallist.append(np.mean(Nintervalsub))
        # Nintervalstd.append(np.std(Nintervalsub))
        lengthlist.append(np.mean(lengthsub))
        lengthstd.append(np.std(lengthsub))
        print('N=',N,',R_0=',R_0)
    fit=op.curve_fit(fitting_func2,Nintervallist,lengthlist,p0=[2,d-1,0])
    mlist.append(fit[0][0])
    mstd.append(fit[1][0,0])
    dlist.append(fit[0][1])
    dstd.append(fit[1][1,1])
    Alist.append(fit[0][2])
    Astd.append(fit[1][2,2])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
N= 100.0 ,R_0= 1e-05
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
N= 150.0 ,R_0= 1e-05
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
N= 200.0 ,R_0= 1e-05
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
N= 250.0 ,R_0= 1e-05
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
N= 300.0 ,R_0= 1e-05
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
N= 350.0 ,R_0= 1e-05
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
N= 400.0 ,R_0= 1e-05
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
N= 450.0 ,R_0= 1e-05
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
N= 500.0 ,R_0= 1e-05
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
N= 550.0 ,R_0= 1e-05
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
N= 600.0 ,R_0= 1e-05
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
N= 650.0 ,R_0= 1e-05
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
N= 700.0 ,R_0= 1e-05
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
N= 750.0 ,R_0= 1e-05
0
1
2


In [204]:
plt.title('Longest chain scaling of dimension with half interval; d=%s, L=%s'%(d,L),fontsize=15)
plt.xlabel(r'$R_0$')
plt.ylabel('scaling dimension')
plt.xscale('log')
plt.grid()
plt.errorbar(R_0list,dlist,yerr=dstd)
plt.show()

In [205]:
plt.title(r'Longest chain scaling of $m_d$ with half interval; d=%s, L=%s'%(d,L),fontsize=15)
plt.xlabel(r'$R_0$')
plt.ylabel(r'$m_d$')
plt.xscale('log')
plt.grid()
plt.errorbar(R_0list,mlist,yerr=mstd)
plt.show()

In [54]:
#longest chain in minkowski space
Nlist=np.linspace(100,1000,20)
L=1
d=2
dlist=[]
dstd=[]
mlist=[]
mstd=[]
Alist=[]
Astd=[]
discretenesslist=[]
interval_centre=np.zeros(d)
V=L**d/2
Nintervallist=[]
Nintervalstd=[]
lengthlist=[]
lengthstd=[]
for N in Nlist:
    density=N/V
    discreteness=density**(-1/d)
    discretenesslist.append(discreteness)
    Nintervalsub=[]
    lengthsub=[]
    for count1 in range(20):
        position=mn.sprinkle(N,L,d)
        position_subset=filter_interval(position,L,interval_centre)
        link=find_link(position_subset)
        longest=longest_chain(link)
        length=len(longest)
        lengthsub.append(length)
        Ninterval=len(position_subset)
        Nintervalsub.append(Ninterval)
        print(count1)
    Nintervallist.append(np.mean(Nintervalsub))
    Nintervalstd.append(np.std(Nintervalsub))
    lengthlist.append(np.mean(lengthsub))
    lengthstd.append(np.std(lengthsub))
    print('N=',N)
fit=op.curve_fit(fitting_func2,Nintervallist,lengthlist,p0=[2,d-1,0])
mlist.append(fit[0][0])
mstd.append(fit[1][0,0])
dlist.append(fit[0][1])
dstd.append(fit[1][1,1])
Alist.append(fit[0][2])
Astd.append(fit[1][2,2])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
N= 100.0
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
N= 147.3684210526316
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
N= 194.73684210526318
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
N= 242.10526315789474
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
N= 289.47368421052636
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
N= 336.8421052631579
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
N= 384.2105263157895
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
N= 431.5789473684211
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
N= 478.94736842105266
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
N= 526.3157894736842
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
N= 573.6842105263158
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
N= 621.0526315789474
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
N= 668.421052631579
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
N= 715.7894736842105
0
1
2
3
4
5
6
7

In [190]:
plt.title(r'Discreteness length for minkowski interval; d=%s L=%s'%(d,L),fontsize=15)
plt.xlabel(r'$\left\langle N_{interval}\right\rangle$')
plt.ylabel(r'$l_{discrete}$')
plt.errorbar(Nintervallist,discretenesslist,xerr=Nintervalstd)
plt.grid()
plt.show()


In [59]:
fit=op.curve_fit(fitting_func2,Nintervallist,lengthlist,p0=[2,d-1,0])
print('m2=',fit[0][0],',d=',fit[0][1],',A=',fit[0][2])
plt.title(r'Minkowski Longest chain length scaling against $\left\langle N_{interval}\right\rangle$; d=%s, L=%s'%(d,L),fontsize=15)
plt.xlabel(r'$\left\langle N_{interval}\right\rangle$')
plt.ylabel('Length of longest chain')
plt.plot(Nintervallist,fitting_func2(Nintervallist,*fit[0]))
plt.errorbar(Nintervallist,lengthlist,yerr=lengthstd,xerr=Nintervalstd)
plt.grid()
plt.show()

m2= 2.174411468821018 ,d= 2.074763208400055 ,A= -3.4893557632215315


In [12]:
#finding m2 in minkowski 2
N=1000
L=1
d=2
repeats=200
interval_centre=np.zeros(d)
constantlist=[]
for count1 in range(repeats):
    position=mn.sprinkle(N,L,d)
    link=find_link(position)
    longest=longest_chain(link)
    length=len(longest)
    constantlist.append(length/(N/2)**(1/d))
    print(count1)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199


In [45]:
mean=np.mean(constantlist)
std=np.std(constantlist)
plt.title(r'Distribution of $m_2$ over %i runs; $\langle N\rangle$=%s d=%s L=%s'%(repeats,N,d,L),fontsize=15)
plt.hist(constantlist,bins=15)
plt.xlabel(r'$m_2$')
plt.ylabel('Occurence')
plt.legend(['Mean=%.2f, Standard deviation=%.2f'%(mean,std)],fontsize=14,loc='lower right')
plt.show()